In [1]:
import scanpy as sc
import scanpy.external as sce

import anndata
from pathlib import Path
from tqdm import tqdm
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [3]:
from scipy import stats

In [4]:
import scipy.sparse as sp
import anndata as ad

In [5]:
import json

import dask.dataframe as dd
import numpy as np
import pandas as pd
#import relation_data_lake as rdl
import scanpy as sc
import scipy.sparse as sp
from anndata import AnnData
#from relation_data_lake.aws.aws_api import pull_folder

In [6]:
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=400, figsize = [1.5*6.4, 1.5*4.8])
sns.set_style("whitegrid")

In [7]:
n_jobs = 50

sc._settings.ScanpyConfig(
    n_jobs=n_jobs
)

In [8]:
import loompy as lp
import json
import zlib
import glob
import base64

/opt/pyenv/versions/3.10.12/lib/python3.10/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/opt/pyenv/versions/3.10.12/lib/python3.10/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dna_to_twobit(dna: str) -> int:
/opt/pyenv/versions/3.10.1

In [9]:
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_rss
import matplotlib.pyplot as plt
from adjustText import adjust_text
import seaborn as sns
from pyscenic.binarization import binarize


# Load data and create objects

In [10]:
adata = sc.read_h5ad('../Rd13_Data_post_filtering_cell_types-reseq.h5ad.gzip')

In [11]:
adata.write_loom('Rd13_Data_post_filtering_cell_types-reseq.loom', write_obsm_varm=True )

In [12]:
f_tfs = "../pySCENIC/mouse/allTFs_mm.txt" 
f_loom_path_scenic = "Rd13_Data_post_filtering_cell_types-reseq.loom"


In [13]:
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata.obs_names) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
}
lp.create( f_loom_path_scenic, adata.X.transpose(), row_attrs, col_attrs)

In [ ]:
! pyscenic grn {f_loom_path_scenic} {f_tfs} -o adjnew.csv --num_workers 8

/opt/pyenv/versions/3.10.12/lib/python3.10/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/opt/pyenv/versions/3.10.12/lib/python3.10/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dna_to_twobit(dna: str) -> int:
/opt/pyenv/versions/3.10.1

In [ ]:
# ranking databases
f_db_glob = "../pySCENIC/mouse/*feather"
f_db_names = ' '.join( glob.glob(f_db_glob) )

# motif databases
f_motif_path = "../pySCENIC/mouse/motifs-v10nr_clust-nr.mgi-m0.001-o0.0.tbl"

In [ ]:
!pyscenic ctx adjnew.csv \
    {f_db_names} \
    --annotations_fname {f_motif_path} \
    --expression_mtx_fname {f_loom_path_scenic} \
    --output reg.csv \
    --mask_dropouts \
    --num_workers 8

In [ ]:
f_pyscenic_output = "pyscenic_output.loom"

In [ ]:
!pyscenic aucell \
    {f_loom_path_scenic} \
    reg.csv \
    --output {f_pyscenic_output} \
    --num_workers 16

In [ ]:
# scenic output
lf = lp.connect( f_pyscenic_output, mode='r', validate=False )
meta = json.loads(zlib.decompress(base64.b64decode( lf.attrs.MetaData )))
exprMat = pd.DataFrame( lf[:,:], index=lf.ra.Gene, columns=lf.ca.CellID).T
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)

In [ ]:
auc_mtx.to_csv('auc_mtx.csv')
auc_mtx

In [ ]:
rss_cellType = regulon_specificity_scores( auc_mtx, adata.obs.cell_type )
rss_cellType

In [ ]:
rss_cellType.to_csv('rss_cellType.csv')

In [ ]:
cats = sorted(list(set(adata.obs.cell_type )))

fig = plt.figure(figsize=(12, 16))
for c,num in zip(cats, range(1,len(cats)+1)):
    x=rss_cellType.T[c]
    ax = fig.add_subplot(4,3,num)
    plot_rss(rss_cellType, c, top_n=10, max_n=None, ax=ax)
    ax.set_ylim( x.min()-(x.max()-x.min())*0.05 , x.max()+(x.max()-x.min())*0.05 )
    for t in ax.texts:
        t.set_fontsize(12)
    ax.set_ylabel('')
    ax.set_xlabel('')
    adjust_text(ax.texts, autoalign='xy', ha='right', va='bottom', arrowprops=dict(arrowstyle='-',color='lightgrey'), precision=0.001 )
 
fig.text(0.5, 0.0, 'Regulon', ha='center', va='center', size='x-large')
fig.text(0.00, 0.5, 'Regulon specificity score (RSS)', ha='center', va='center', rotation='vertical', size='x-large')
plt.tight_layout()
plt.rcParams.update({'figure.autolayout': True, 'figure.titlesize': 'medium' ,'axes.labelsize': 'medium','axes.titlesize':'medium','xtick.labelsize':'medium','ytick.labelsize':'medium'})
plt.savefig("figures/Fibroblast_perturb-seq_cellType-RSS-top10.pdf", dpi=600, bbox_inches = "tight")
plt.savefig("figures/Fibroblast_perturb-seq_cellType-RSS-top10.png", dpi=600, bbox_inches = "tight")
plt.savefig("figures/Fibroblast_perturb-seq_cellType-RSS-top10.svg", dpi=600, bbox_inches = "tight")

plt.show()

In [ ]:
sns.histplot(rss_cellType.max(), bins = 50)

In [ ]:
rss_cellType.drop(columns=rss_cellType.columns[rss_cellType.isna().any()], inplace=True)

In [ ]:
#sns.clustermap(rss_cellType)

g = sns.clustermap(rss_cellType.drop(columns = rss_cellType.columns[rss_cellType.max() < 0.25] ),
               #row_linkage=row_linkage_df, 
               #col_linkage=col_linkage_df, 
               figsize=(24, 6),
               cmap="binary",
               row_cluster = True, 
               col_cluster = True,
               #metric='ward',
               vmin = 0.25, vmax = 0.4,
               dendrogram_ratio=[0.03, 0.2],
               xticklabels =1, yticklabels=1,
               #linewidth=0.1, linecolor='w',
              )
#ax.hlines([3, 6, 9], *ax.get_xlim())
#g.cax.set_visible(False)
#g.ax_row_dendrogram.set_visible(False)
#g.ax_col_dendrogram.set_visible(False)

plt.savefig("figures/clustermap_RSS1.pdf", dpi=600, bbox_inches = "tight")
plt.savefig("figures/clustermap_RSS1.svg", dpi=600, bbox_inches = "tight")
plt.savefig("figures/clustermap_RSS1.png", dpi=600, bbox_inches = "tight")